In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import datetime
%matplotlib inline 

In [ ]:
# load the data
path_ed = '/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/'
path_m = '/media/kpierce/seagate/reverting-onecell/analysis/m-stats/'
path_cdf = '/media/kpierce/seagate/reverting-onecell/analysis/rt-cdf/'
efile = 'flow_a-l_0.6-time_1000.0hr-entrains.npy'
dfile = 'flow_a-l_0.6-time_1000.0hr-deposits.npy'
mfile = 'flow_a-l_0.6-time_1000.0hr-mstats.dat'
cfile = 'flow_a-l_0.6-time-1000.0hr-rtcdf.npy'
def loader(path_ed, path_m, path_cdf, efile, dfile, mfile):
    te, me = np.load(path_ed+efile).T
    td, md = np.load(path_ed+dfile).T
    m, pm  = np.loadtxt(path_m+mfile).T

In [ ]:
# define parameters
tmax = 500*3600 # maximum return time to allow for (seconds) 
dt = 0.5 # bin size in seconds
bins = np.arange(0, tmax, dt) # bins over which to histogram

In [ ]:
def compute_crt(te, me, td, md, mv, bins = bins):
    """ 
    compute conditional return time given
    - entrainment timeseries te
    - bed elevations me at entrainment
    - deposition timeseries td
    - bed elevations md at deposition
    - a value mv at which to condition
    - a set of bins in time over which to histogram 
    """
    returns = te[me==mv]
    departs = td[md==mv+1]
    
    if (len(returns)>0) and (len(departs)>0): 
        while (returns[0]<departs[0]):
            returns=returns[1:]
        while (departs[-1]>returns[-1]):
            departs=departs[:-1]
        k = len(departs)-len(returns)
        if k>0:
            departs=departs[k:]
        elif k<0:
            returns=returns[:k]
        crt = returns-departs
        H,_ = np.histogram(crt,bins=bins,density=True)
        F = 1-H.cumsum()*(bins[1]-bins[0])
    else:
        F = np.zeros(len(bins)-1,dtype=float)
    return F

In [ ]:
def compute_cdf(cdf_filename, bins=bins):
    cdf = np.zeros(len(bins)-1,dtype=float) # fill this iteratively
    i = 0 
    for mv,p in zip(m, pm): # iterate through all mvalues in question
        i+=1
        Fmv = compute_crt(te, me, td, md, mv)
        cdf+=Fmv*p
    bins = (bins[1:]+bins[:-1])/2.0 # compute the midpoints of the bins 
    data = np.array((bins, cdf)).T
    np.save(path_cdf+cfile, data)
    del data

In [ ]:
path_m = '/media/kpierce/seagate/reverting-onecell/analysis/m-stats/'
path_ed = '/media/kpierce/seagate/reverting-onecell/analysis/entrain-deposit/'
path_c = '/media/kpierce/seagate/reverting-onecell/analysis-codes/rt-cdf/'
nfiles = len(list(os.listdir('/media/kpierce/seagate/reverting-onecell/analysis/m-stats/')))
i=0
print('------------------------------------------------------------------------------')
for mfile in os.listdir('/media/kpierce/seagate/reverting-onecell/analysis/m-stats/'):
    i+=1
    print('starting ' + mfile[:-10] + ' at ' + datetime.datetime.utcnow().strftime("%H:%M:%S"))
    print('computation {} of {}.'.format(i,nfiles))
    efile = path_ed+mfile[:-10]+'entrains.npy'
    dfile = path_ed+mfile[:-10]+'deposits.npy'
    cfile = path_c+mfile[:-10]+'rtcdf'
    te, me = np.load(efile).T
    td, md = np.load(dfile).T
    m, pm  = np.loadtxt(path_m+mfile).T
    compute_cdf(cfile)
    print('finished at ' + datetime.datetime.utcnow().strftime("%H:%M:%S"))
    print('------------------------------------------------------------------------------')
